In [147]:
import ast
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

In [148]:
df_train = pd.read_csv("datasets/x_train.csv", index_col=None)
df_train.job_ids = df_train.job_ids.apply(ast.literal_eval)
df_train.actions = df_train.actions.apply(ast.literal_eval)
df_train.head()

,session_id,job_ids,actions
0,0,"[305, 299, 300, 290, 282, 274, 264, 261]","[view, view, view, view, view, view, view, view]"
1,1,"[84, 257, 252, 250]","[view, view, view, view]"
2,2,"[241, 237, 221, 309, 310, 306, 301]","[view, view, apply, apply, apply, apply, apply]"
3,3,"[303, 297, 296, 298, 294, 295, 292, 293]","[apply, apply, apply, apply, apply, apply, app..."
4,4,"[171, 291, 289, 166, 288, 155]","[apply, apply, apply, apply, apply, apply]"


In [149]:
job_ids = [set(job_list) for job_list in df_train.job_ids]
job_ids = set.union(*job_ids)
jod_id_to_idx = {id: idx for idx, id in enumerate(job_ids)}
jod_idx_to_id = {idx: id for id, idx in jod_id_to_idx.items()}

In [150]:
df_train.job_ids = df_train.job_ids.apply(lambda ids : [jod_id_to_idx[id] for id in ids])

In [151]:
def indices_to_binary_vector(indices):
    binary_vector = np.zeros(len(job_ids))
    binary_vector[indices] = 1
    return binary_vector

df_train.job_ids = df_train.job_ids.apply(indices_to_binary_vector).to_list()

In [152]:
K = 20
interaction_matrix = np.array(df_train.job_ids.to_list())
similarity_matrix = cosine_similarity(interaction_matrix)
similar_users = np.argsort(similarity_matrix, axis=1)[:, -K-1:-1][:, ::-1]

In [153]:
# transform the indices into binary vector
vectorized_similar_users = np.zeros((interaction_matrix.shape[0], interaction_matrix.shape[0]))
np.put_along_axis(vectorized_similar_users, similar_users, 1, axis=1)
job_popularity = vectorized_similar_users @ interaction_matrix / K
popular_jobs = np.argsort(job_popularity, axis=1)[:, -10:][:, ::-1]
sorted_popularity = np.take_along_axis(job_popularity, popular_jobs, axis=1)